In [9]:
import tensorflow as tf

grad_clip = 5

In [10]:
class LinearRegression(tf.keras.Model):
  def __init__(self):
    super(LinearRegression, self).__init__()
    self.linear_layer = tf.keras.layers.Dense(1, activation=None)

  def call(self, x):
    y_pred = self.linear_layer(x)

    return y_pred

In [13]:
@tf.function
def mse_loss(y_pred, y):
  return tf.reduce_mean(tf.square(y_pred -y))

optimizer = tf.optimizers.SGD(0.01)

In [21]:
@tf.function
def train_step(model, x, y):
  with tf.GradientTape() as tape:
    y_pred = model(x)
    loss = mse_loss(y_pred, y)
  gradients = tape.gradient(loss, model.trainable_variables)

  clipped_grads = []
  for grad in gradients:
    clipped_grads.append(tf.clip_by_norm(grad, grad_clip))
  optimizer.apply_gradients(zip(clipped_grads, model.trainable_variables))

In [22]:
x_train = [1.0, 2.0, 3.0, 4.0]
y_train = [2.0, 4.0, 6.0, 8.0]

train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().batch(1)
train_data_iter = iter(train_data)

In [23]:
LinearRegression_model = LinearRegression()

In [24]:
for i in range(1000):
  batch_xs, batch_ys = next(train_data_iter)

  batch_xs = tf.expand_dims(batch_xs, 0)
  train_step(LinearRegression_model, batch_xs, batch_ys)

In [25]:
x_test = [3.5, 5.0, 5.5, 6.0]
test_data = tf.data.Dataset.from_tensor_slices((x_test))
test_data = test_data.batch(1)

In [26]:
for batch_x_test in test_data:
  batch_x_test = tf.expand_dims(batch_x_test, 0)

  print(tf.squeeze(LinearRegression_model(batch_x_test), 0).numpy())

[6.9882345]
[9.925688]
[10.9048395]
[11.883991]
